In [9]:
import os, sys, subprocess, textwrap, pathlib, time, threading, signal

PROJECT_ROOT = pathlib.Path('.').resolve()
FRONTEND_DIR = PROJECT_ROOT / 'frontend'

print('Project root:', PROJECT_ROOT)
print('Frontend dir exists:', FRONTEND_DIR.exists())
print('Python executable:', sys.executable)


Project root: /Users/elizabethsu/CS 410/ReadMatch
Frontend dir exists: True
Python executable: /usr/local/bin/python3


## 1) Install Python dependencies

In [10]:
req = PROJECT_ROOT / 'requirements.txt'

if req.exists():
    print('Installing from requirements.txt...')
    # subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-r', str(req)])
    req = PROJECT_ROOT / 'requirements.txt'

else:
    print('No requirements.txt found. Installing a minimal set of common deps...')
    minimal = [
        'flask',
        'flask-cors',
        'pandas',
        'numpy',
        'sqlalchemy',
        'scikit-learn',
        'tqdm',
        'requests'
    ]
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', *minimal])

print('Python deps installed.')


No requirements.txt found. Installing a minimal set of common deps...
Python deps installed.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip3 install --upgrade pip


## 2) Install Node dependencies

In [11]:
pkg = FRONTEND_DIR / 'package.json'
if not FRONTEND_DIR.exists():
    raise FileNotFoundError('frontend/ directory not found. Make sure you run this notebook from the project root.')
if not pkg.exists():
    raise FileNotFoundError('frontend/package.json not found.')

print('Installing npm dependencies...')
subprocess.check_call(['npm', 'install'], cwd=str(FRONTEND_DIR))
print('Node deps installed.')


Installing npm dependencies...

up to date, audited 1317 packages in 2s

267 packages are looking for funding
  run `npm fund` for details

9 vulnerabilities (3 moderate, 6 high)

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
Node deps installed.


## 3) Start backend + frontend (concurrently)

In [12]:
backend_proc = None
frontend_proc = None

def _stream(prefix, pipe):
    try:
        for line in iter(pipe.readline, ''):
            if not line:
                break
            print(f'[{prefix}] {line}', end='')
    finally:
        try:
            pipe.close()
        except Exception:
            pass

def start_servers():
    global backend_proc, frontend_proc
    
    if backend_proc and backend_proc.poll() is None:
        print('Backend already running.')
    else:
        print('Starting backend: python3 app.py')
        backend_proc = subprocess.Popen(
            [sys.executable, 'app.py'],
            cwd=str(PROJECT_ROOT),
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            bufsize=1,
        )
        threading.Thread(target=_stream, args=('backend', backend_proc.stdout), daemon=True).start()
    
    if frontend_proc and frontend_proc.poll() is None:
        print('Frontend already running.')
    else:
        print('Starting frontend: npm start')
        env = os.environ.copy()
        # Prevent CRA from trying to be interactive in some notebook terminals
        env.setdefault('CI', 'false')
        frontend_proc = subprocess.Popen(
            ['npm', 'start'],
            cwd=str(FRONTEND_DIR),
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            bufsize=1,
            env=env,
        )
        threading.Thread(target=_stream, args=('frontend', frontend_proc.stdout), daemon=True).start()

start_servers()
print('Servers launching...')
print('Next, run the "Open the app" cell.')


Starting backend: python3 app.py
Starting frontend: npm start
Servers launching...
Next, run the "Open the app" cell.


## 4) Open the app

In [13]:
import webbrowser, time

# Give the dev servers a brief moment to start.
time.sleep(3)

FRONTEND_URL = 'http://localhost:3000'
print('Opening:', FRONTEND_URL)
webbrowser.open(FRONTEND_URL)


[frontend] 
[frontend] > frontend@0.1.0 start
[frontend] > react-scripts start
[frontend] 
[backend] Traceback (most recent call last):
[backend]   File "/Users/elizabethsu/CS 410/ReadMatch/app.py", line 7, in <module>
[backend]     import retrieval
[backend]   File "/Users/elizabethsu/CS 410/ReadMatch/retrieval.py", line 1, in <module>
[backend]     from rank_bm25 import BM25Okapi
[backend] ModuleNotFoundError: No module named 'rank_bm25'
[frontend] Something is already running on port 3000.


## 5) Stop servers


In [ ]:
def _terminate(proc, name):
    if not proc:
        print(f'{name}: not started.')
        return
    if proc.poll() is not None:
        print(f'{name}: already stopped.')
        return
    print(f'Stopping {name}...')
    try:
        # Try graceful terminate first
        proc.terminate()
        try:
            proc.wait(timeout=5)
        except Exception:
            proc.kill()
    except Exception as e:
        print(f'Error stopping {name}:', e)

_terminate(frontend_proc, 'frontend')
_terminate(backend_proc, 'backend')
